In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import re
import nltk
from nltk.corpus import stopwords

In [2]:
USER_AGENT = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
              'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8'}
STOPWORDS = set(stopwords.words("portuguese"))

In [3]:
sites = pd.read_json('./Links/Sites.json')
stores = sites.columns
sites

,Gregory,Damyller,Dassi,Excluiva,CutiCutiBaby,BluK,Villevie,Achados96,DWZ,BellaSeda
negative,[https://www.gregory.com.br/institucional/sobr...,[https://www.damyller.com.br/boho-denim?O=Orde...,"[https://www.boutiquedassi.com.br/moda-praia, ...",[https://www.lojasexclusiva.com.br/shorts/Shor...,[https://www.cuticutibaby.com.br/vestido-gola-...,[https://www.bluk.com.br/vestido-longo-crepe-a...,[https://www.lojavillevie.com.br/kaftan-listra...,[https://www.achados96.com.br/acessorios/cinto...,[https://www.dwz.com.br/saia-jeans-barra-desfi...,[https://www.bellaseda.online/produto/289491/c...
positive,[https://www.gregory.com.br/blusa-cachecouer-m...,[https://www.damyller.com.br/blusa-decote-vaza...,[https://www.boutiquedassi.com.br/BLUSA-MANGA-...,[https://www.lojasexclusiva.com.br/t-shirts/T-...,[https://www.cuticutibaby.com.br/t-shirt-stree...,[https://www.bluk.com.br/bata-visco-linho-com-...,[https://www.lojavillevie.com.br/top-blusa/man...,[https://www.achados96.com.br/todos/t-shirt-ov...,[https://www.dwz.com.br/blusa-manga-longa-tule...,[https://www.bellaseda.online/produto/254694/b...


In [4]:
data = []
for store in stores:
    for i in range(0,2):
        for url in sites[store][i]:
            data.append([url, i])
df = pd.DataFrame(data, columns = ['url', 'label'])
df.head()

,url,label
0,https://www.gregory.com.br/institucional/sobre,0
1,https://www.gregory.com.br/promocoes,0
2,https://www.gregory.com.br/calca-boot-cut-esta...,0
3,https://www.gregory.com.br/institucional/troca...,0
4,https://www.gregory.com.br/colar-curto-pedrasc...,0


In [5]:
def clean(text):
    d = re.sub("[^a-zA-Z]", " ", text)
    words = d.lower().split()
    words = [w for w in words if w not in STOPWORDS]
    words = sorted(list(set(words)))
    return words

In [6]:
def content_page(page):
    req = requests.get(url, headers=USER_AGENT)
    content = req.text
    s = BeautifulSoup(content, 'html.parser')
    data = s.find_all(text=True)
    data = ''.join(data)
    text = data.replace('\n', ' ').replace('\r', '').strip()
    text = ' '.join(text.split())
    return clean(text)

In [9]:
dataset = []
for line in df.values:
    print("Lendo " + line[0])
    content = content_page(line[0])
    dataset.append([content, line[1]])
    time.sleep(1)
dataset = pd.DataFrame(dataset, columns=["content", "label"])

Lendo https://www.gregory.com.br/institucional/sobre
Lendo https://www.gregory.com.br/promocoes
Lendo https://www.gregory.com.br/calca-boot-cut-estampa-floral-verde-02-78-0366-024/p
Lendo https://www.gregory.com.br/institucional/troca-loja-virtual
Lendo https://www.gregory.com.br/colar-curto-pedrascoral-28-07-0089-070/p
Lendo https://www.gregory.com.br/colar-curto-perolas-e-corrente-dourado-28-06-0069-134/p
Lendo https://www.gregory.com.br/colar-curto-elos-resina-e-brilho-preto-28-06-0070-002/p
Lendo https://www.gregory.com.br/novidades?O=OrderByReleaseDateDESC?O=OrderByReleaseDateDESC
Lendo https://www.gregory.com.br/vestido-bicolor-preto-off-white-08-23-0125-196/p
Lendo https://www.gregory.com.br/calca-cigarrete-xadrezmarrom-02-10-0758-043/p
Lendo https://www.gregory.com.br/roupas-femininas/macacao?O=OrderByReleaseDateDESC?O=OrderByReleaseDateDESC
Lendo https://www.gregory.com.br/macacao-decote-voff-white-22-02-0029-175/p
Lendo https://www.gregory.com.br/shorts-com-botoes-off-white-2

Lendo https://www.cuticutibaby.com.br/t-shirt-raglan-long-ufofo
Lendo https://www.cuticutibaby.com.br/regata-street-pinguinhos
Lendo https://www.cuticutibaby.com.br/regata-street-flechas
Lendo https://www.cuticutibaby.com.br/camisa-kids-amigos-no-bosque
Lendo https://www.cuticutibaby.com.br/camisa-frufru-manga-essential
Lendo https://www.cuticutibaby.com.br/blusao-conforto-plush-capuz
Lendo https://www.cuticutibaby.com.br/camiseta-manga-longa-back
Lendo https://www.bluk.com.br/vestido-longo-crepe-alcas-com-macrame-preto--vt60740719_265/p
Lendo https://www.bluk.com.br/calca-reta-linho-lateral-com-lateral-tecido-bordado-off-limoncello-ct61160719_729/p
Lendo https://www.bluk.com.br/calca-reta-malha-estampa-monterosso-estampado--cm62460819_499/p
Lendo https://www.bluk.com.br/vestido-midi-tecido-acetinado-estampado-frente-com-transpasse-e-amarracao--estampado--vt57420519_499/p
Lendo https://www.bluk.com.br/casaqueto-linho-com-tecido-bordado-e-bico-off-limoncello-zt61180719_729/p
Lendo https

Lendo https://www.dwz.com.br/blusa-cropped-ombro-a-ombro-poa-190804015-p1143527?pp=/44.573/
Lendo https://www.dwz.com.br/blusa-lisa-ombros-a-mostra-181104028-p985198?pp=/44.1398/
Lendo https://www.dwz.com.br/blusa-ombro-aberto-com-tiras-181104027-p991613?pp=/44.573/
Lendo https://www.dwz.com.br/blusa-oncinha-190404045-p1000794?pp=/44.614/
Lendo https://www.dwz.com.br/blusa-plus-estampada-floral-181004010-p1019090?pp=/44.702/
Lendo https://www.dwz.com.br/blusa-sem-manga-gola-alta-190804011-p1143337?pp=/44.1398/
Lendo https://www.dwz.com.br/blusa-moletom-paris-190404010-p987368?pp=/44.545/
Lendo https://www.bellaseda.online/produto/289491/calca-jogger
Lendo https://www.bellaseda.online/produto/274652/vestido-tubinho
Lendo https://www.bellaseda.online/produto/283088/calca-suede
Lendo https://www.bellaseda.online/produto/288624/bermuda-fardamento
Lendo https://www.bellaseda.online/produto/261188/macacao-estampado
Lendo https://www.bellaseda.online/produto/251043/macacao-crepe-scuba-pantaco

,0,1
0,"[active, adicionar, ainda, amento, analytics, ...",0
1,"[active, adicionar, ainda, amento, analytics, ...",0
2,"[active, adicionar, ainda, amento, analytics, ...",0
3,"[active, adicionar, ainda, amento, analytics, ...",0
4,"[active, adicionar, ainda, amento, analytics, ...",0


In [10]:
dataset.head()

,0,1
0,"[active, adicionar, ainda, amento, analytics, ...",0
1,"[active, adicionar, ainda, amento, analytics, ...",0
2,"[active, adicionar, ainda, amento, analytics, ...",0
3,"[active, adicionar, ainda, amento, analytics, ...",0
4,"[active, adicionar, ainda, amento, analytics, ...",0
